<a href="https://colab.research.google.com/github/za98/CS301_Project_AI/blob/main/Group_3_(Matt_and_Ziad)_Accelerated_Functional_Programming_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Contradictory, My Dear Watson

---

## **Group 3 Members**

1. Matthew Barrera
2. Ziad Abou-Ghali

## **Important Notice**

> After the split of Group 3, Ziad and I (Matthew) still have Steps 1-3 complete since we all contributed to that. We are re-starting Step 4 after Omar split with the code. Furthermore, we will create a new Github for our group to represent just us two. Again, the proposal is still the same since all of us contribted equally to that portion.

> Essentially, the differences between our projects include: the code, the research, and the final report.

## **Goal**

> Classifying pairs of sentences (Premise and Hypothesis) into three categories of "Entailment, Contradiction, Neutral" utilizing JAX. For more information about the specifics of the task, visit link 3 (Kaggle Competition).


> Link 4 (Tutorial Notebook) has plenty of valuable information about this type of problem, but the only imports allowed are jax and pandas (Numpy can be obtained directly using jax). All other imports are not allowd for this project.

## **Relevant Links**

1. **[Project Instructions P1](https://pantelis.github.io/data-science/aiml-common/projects/jaxworld/jax.html)**

2. **[Project Instructions P2](https://pantelis.github.io/data-science/aiml-common/projects/_index.html)**

3. **[Kaggle Competition](https://www.kaggle.com/competitions/contradictory-my-dear-watson/overview)**

4. **[Kaggle Tutorial Notebook Inspiration](https://www.kaggle.com/code/anasofiauzsoy/tutorial-notebook/notebook)**

5. **[Project GitHub Page](https://github.com/za98/CS301_Project_AI)**

6. **[Project Report](https://docs.google.com/document/d/10kCexSI3Ur2R15ld3m_vKeyWFI16broqtUQDaWzSwAU/edit?usp=sharing)**

### **Preparing the Data**

*Steps 1 - 3.5*

---

Before the a training model can be created, all necessary packages should be imported that are needed to properly initialize the datasets.

In [ ]:
#Step 1: Import jax and jax.numpy and pandas
       # This can only be run once; the !pip is used to ensure there are no issues in utilizing any package

!pip install jax
!pip install pandas
!pip install numpy

import jax
import jax.numpy as jnp
import numpy as np #Necessary for the encoder --> JAX cannot handle this
import pandas as pd

In [ ]:
#Step 1.5: Confirm version of packages to ensure which version is being used (This may impact functions used slightly, but we need to make sure)
print(jax.__version__)
print(pd.__version__)

0.3.4
1.3.5


In [ ]:
#https://towardsdatascience.com/7-ways-to-load-external-data-into-google-colab-7ba73e7d5fc7
       # Method 4 was used
#Step 2: Clone the GitHub repository into the Colab environment
       # This repository be seen in link 5 (Project GitHub Page)
       # This step is done to gain local access to train.csv and test.csv needed to reach the goal
       # This can only be run once; once CS301_Project_AI is filled with data, it will give an error when the code is run again

!git clone https://github.com/za98/CS301_Project_AI

Cloning into 'CS301_Project_AI'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 15 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (15/15), done.


In [ ]:
#Same link and method used as in Step 2
#Step 3: Take files imported into GitHub repository and read them in using pd

trainData = pd.read_csv("CS301_Project_AI/train.csv")
testData = pd.read_csv("CS301_Project_AI/test.csv")

In [ ]:
#Step 3.5: Confirm the sizes of the trainData and testData - Ensure that we have the full information to analyze

len(trainData["id"].unique())

12120

In [ ]:
#Step 3.5 Continued: Same as above --> Outputs of len conflict if in the same code block

len(testData["id"].unique())

5195

### **Tokenzing the Data (Creating Training Model Part 1)**

*Steps 4 - 8*

---

Now that the stage has been set, we can finally begin to create a training model. This involves a new set of imports needed to tokenize the data, since this is one of the more efficient ways of converting the data from words to numbers.

In [ ]:
#Step 4: Import BertTokenizer and TFBertModel from transformers
       # This can only be run once; the !pip is used to ensure there are no issues in utilizing any package

!pip install transformers
!pip install tensorflow

# https://huggingface.co/docs/transformers/main/en/model_doc/bert#transformers.BertTokenizer
# ^ used to assemble the input that a build model would require
# https://huggingface.co/docs/transformers/main/en/model_doc/bert#transformers.TFBertModel
# ^ takes those assembled inputs and builds a model

from transformers import BertTokenizer, TFBertModel
import tensorflow as tf

     |████████████████████████████████| 4.0 MB 5.3 MB/s 
     |████████████████████████████████| 6.6 MB 7.2 MB/s 
     |████████████████████████████████| 596 kB 40.3 MB/s 
     |████████████████████████████████| 77 kB 3.9 MB/s 
     |████████████████████████████████| 895 kB 44.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 462 kB 5.2 MB/s 


In [ ]:
#A https://www.kaggle.com/code/anasofiauzsoy/tutorial-notebook/notebook
#B https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/tokenizer_training.ipynb#scrollTo=a6BIAHQogpT0
#Step 5: Use a pretrained model as a basis to analyze multilingual information
       # Both links reference similar ways to tokenize the data, we will alternate between the two documents drawing what we need from each
       # This particular step draws more from the A link
       # As mentioned earlier, the tokenizer will turn our word datasets into numbers


model_name = 'bert-base-multilingual-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)

#https://github.com/google-research/bert/blob/master/multilingual.md
#The model name chosen is super important --> We specifically want a pre-trained mutilingual model
#The kaggle link goes with the cased version since its the latest and recommended version --> We will follow suit
#The cased version includes 104 languages, 12-layers, 768-hidden layers, 12-heads, and 110M parameters --> More than enough for Kaggle project's numerous languages in datasets

Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [ ]:
#Step 6: Create function to insert a "separation" (in form of [SEP]) between each premise and hypothesis pair that are being compared/evaluated
       # [SEP] token in encode_sentence(s) represents a separation between a PREMISE and HYPOTHESIS
       # Draws more from the A link in Step 5 section

#The argument "s" represents a line from the dataset that is currently being read
#Thus, the line is read and put into a table (Utilizing jnp) with [premise, hypothesis] structure
#Then, the [SEP] token is inserted for the reasons mentioned above
#Finally, we return either an int or a list of ints

def encode_sentence(s):
   tokens = list(tokenizer.tokenize(s))
   tokens.append('[SEP]')
   return tokenizer.convert_tokens_to_ids(tokens)

In [ ]:
#Step 7: Create a function that accepts data from a dataset ann transforms it into numbers
       # Will accept all of the hypothesis-premise pairings and use the tokenizer to tokenize the data (which stands for making the data into nums)
       # Makes reference to encode_sentence(s) to take each individual pairing and add [SEP] between them (makes looping easier for large datasets)
       # Draws a huge amount from the A link in Step 5 section
       # It is expected for the creation of the training model that the trainData be entered into this function, not testData

# An issue our group saw with the original notebook was how the order of parameter was hyp, prem, then tokenizer
# However, whenever bert_encode was called, the values entered were bert_encode(prem.vals, hyp.vals, tokenizer)
# Essential, the premise values were being submitted where the hypothesis values were supposed to go
# Our group made the decision to change our bert_encode call to properly reflect the correct parameters
def bert_encode(hypotheses, premises, tokenizer):

  num_examples = len(hypotheses) #Hypothesis:Premise ratio is 1:1 <-- Thus, this tells us the total num of pairs to look at

  # The original code used tensorflow for access to tf.ragged.constant <-- This is necessary to the code
  #Jax does not have ragged arrays and has extreme difficulty with dynamic arrays if we constantly update them
  #Dynamic arrays are possible --> https://jax.readthedocs.io/en/latest/_autosummary/jax.lax.dynamic_slice.html
  #However, to ensure no problems occur when using pmap later, we will utilize tensorflow this it is best designed for training models

  # We cannot use jnp in place of np for this code --> JNP only works for int types --> dtypes (strings) not supported using jax
  # Once again, tensorflow is pretty vital to the project since it allows for variable sized hypothesis and premises to be stored in same array
  # Not having tensorflow makes this task extremely difficult
  sentence1 = tf.ragged.constant([
      encode_sentence(s)
      for s in np.array(hypotheses)])
  sentence2 = tf.ragged.constant([
      encode_sentence(s)
      for s in np.array(premises)])
  
  # The recommendation to use to_tensor(shape=...) came from comments and is needed to fix issues in Step 11

  # This puts a [CLS] symbol at the start of every hypothesis <-- This shows when a new pair is being read
  # We will then take the [CLS] before the Hypothesis and concatenate it with Hypthesis + [SEP] + Premise
  cls = [tokenizer.convert_tokens_to_ids(['[CLS]'])]*sentence1.shape[0]
  input_word_ids = tf.concat([cls, sentence1, sentence2], axis=-1)

  # Although we accepted variable-sized sentences earlier, we need to ensure that the inputs are the same size
  # As such, the input_mask will be created in which all the tensors with the same shape are merged into one tensor
  input_mask = tf.ones_like(input_word_ids).to_tensor(shape=[input_word_ids.shape[0], 50])

  # More tensor manipulation below, since it is a really good method to group the hyp/prem pairs
  type_cls = tf.zeros_like(cls)
  type_s1 = tf.zeros_like(sentence1)
  type_s2 = tf.ones_like(sentence2)
  input_type_ids = tf.concat(
      [type_cls, type_s1, type_s2], axis=-1).to_tensor(shape=[input_word_ids.shape[0], 50])

  # Finally, we are stucturing the return
  # Input word ids - Allows us to identify distinct sentences, most prominently through use of [CLS] and [SEP] tokens
  # Input mask     - Allows us to disregard differences in size between hypothesis and premise pairings --> The pairs with same size
  # Input type ids - Allows us to do the same as input mask, but instead --> The pairs with differing size that need padding
  inputs = {
      'input_word_ids': input_word_ids.to_tensor(shape=[input_word_ids.shape[0], 50]),
      'input_mask': input_mask,
      'input_type_ids': input_type_ids}

  return inputs

In [ ]:
#Step 8: Send the trainData hypothesis and premise into the bert_encode function to transform the data
       # The tokenizer should also be given so that the pretrained multilingual data can help parse through the hyp/prem

# This section of code is not suitable for a jax operation
# As mentioned in step 7, jnp cannot be used for the array since hypothesis and premises have to be processed as the words that they are
# When it comes to jax.jit, trainData's hyp and prem iterate from the 1st value to the last value, which means that values aren't static
# There is also the issue in that jit likes to work with integers, and non-integers are being submitted
# This will raise an error, so in order to turn the data in ints, we cannot use jit which requires the data to already be an int (not possible)

trainDataEncoded = bert_encode(trainData.hypothesis.values, trainData.premise.values, tokenizer)

# https://jax.readthedocs.io/en/latest/notebooks/thinking_in_jax.html#to-jit-or-not-to-jit
# https://jax.readthedocs.io/en/latest/notebooks/thinking_in_jax.html#jit-mechanics-tracing-and-static-variables
# These two links exist on same page, but represent different subsections
# Better showcases what I am describing about the code up until this point
# We will see for the future code segments which one can fully utilize jax

### **Fitting the Tokens to a Model (Creating the Training Model Part 2)**

*Steps 9 - 12*

---

Once the tokens from the trainData dataset have been created, we can combine that information together to create the training model. We can start utilizing Jax to a higher degree now that the data has become an integer.

In [ ]:
#Step 9: Create a function that will define how to build a training model

max_len = 50

def build_model():
    # The same model name will be used as was for BertTokenizer
    # However, instead of BertTokenizer.from_pretrained(model_name)...
    # ...we want to use TFBertModel.from_pretrained(model_name)

    bert_encoder = TFBertModel.from_pretrained(model_name)

    # Now, all three input's created from bert_encode will be input into tensorflow's keras.Input
    # The standard tensor's will become keras tensors
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    input_type_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_type_ids")
    
    # Based on the research, bert_encoder is called upon the keras tensor versions of the output of original bert_encoder call in step 7
    # This allows their to be more layers in the training model so that it can detect more complex patterns in the input
    embedding = bert_encoder([input_word_ids, input_mask, input_type_ids])[0]
    output = tf.keras.layers.Dense(3, activation='softmax')(embedding[:,0,:])
    # By this point, layers.Dense connected the neural network that constitutes the training model
    
    # Now that the neural network is connected, create the training model using the three given inputs and some output
    model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=output)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # In the training model compilation, the Adam optimization is run --> https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam
    # This is a stochastic gradient descent method based on adaptive estimation of first and second order moments
    
    # Finally, return the building model created
    return model

In [ ]:
#Step 10: Create a strategy object
        # A TPU has a few lines that need to be run to create a strategy, for a CPU only 1 is needed
#https://www.tensorflow.org/api_docs/python/tf/distribute/experimental/TPUStrategy


# THE NOTEBOOK HAS ENABLED TPU
# IF IT IS NOT ENABLED IN YOUR RUNTIME, PLEASE DO SO
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)

#https://www.kaggle.com/code/katharinamenz/tutorial-notebook-edited-by-katharinamenz/notebook
# Very similar notebook to the tutorial notebook linked above
# This one includes more detailed notes, and mentions how TPU is almost essential for this type of problem

#strategy = tf.distribute.get_strategy()
# This was initially used for CPU <-- When run in the program, 1 epoch was clocking with an ETA of 1:20:22
# I don't know if there was an issue with the CPU, but the only realistic way to use strategy is with an accelerated TPU
# As such, we will be using a TPU so that we and anyone that views the code can realistically perform the line

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.41.76.234:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.41.76.234:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
#Step 10: Building the training model use the previous function in step 9

with strategy.scope():
   model = build_model()
   model.summary()

# Once again, Jax cannot be used within strateg.scope() because it interfeers with the model definition
# This means that if jax.jit or jax.grad were used on build_model(), model would no longer have the .summary() or .fit() methods

Downloading:   0%|          | 0.00/1.01G [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 50)]         0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 50)]         0           []                               
                                                                                                  
 input_type_ids (InputLayer)    [(None, 50)]         0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  177853440   ['input_word_ids[0][0]',         
                                thPoolingAndCrossAt               'input_mask[0][0]',         

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
#Step 11: Take the built training model and place the data found in Step 8 into the model
        # Reminder, the data found in step 8 was called trainDataEncoded

# In addition to giving the model the encoded trainData, give it the necessary labels related to each encoded piece of data
# This is important because the trainData has pre-defined values of 0, 1, or 2 based on entailment, contradiction, or neutrality
# These known values will help us determine future decisions of entailment, contradiction, or neutrality
# Furthermore, define a few variables about the training model

model.fit(trainDataEncoded, trainData.label.values, epochs = 2, verbose = 1, batch_size = 64, validation_split = 0.2)

# As mentioned above when creating a strategy using TPU, it was not realistic to use CPU for this project
# CPU offered a time for 1 epoch of over an hour
# Meanwhile, Epoch 1 under a TPU took 174s (almost 3 minutes) and Epoch 2 took 15s

Epoch 1/2


152/152 [==============================] - 182s 522ms/step - loss: 1.0651 - accuracy: 0.4412 - val_loss: 0.9419 - val_accuracy: 0.5470
Epoch 2/2
152/152 [==============================] - 16s 102ms/step - loss: 0.8742 - accuracy: 0.6059 - val_loss: 0.8862 - val_accuracy: 0.5941


In [ ]:
#Step 12: Send the trainData hypothesis and premise into the bert_encode function to transform the data
       # The tokenizer should also be given so that the pretrained multilingual data can help parse through the hyp/prem
       # Very similtar to step 8 and how trainData was encoded

# Don't forget that testData was opened when trainData was opened way back at Step 3 --> No need to open it here like in Tutorial Notebook
testDataEncoded = bert_encode(testData.hypothesis.values, testData.premise.values, tokenizer)

# Once again, we chose to go hyp then prem then tokenizer as opposed to the tutorial notebook
# Since this way, it actually matches the functions parameters

### **Predicting Test Data & Comparing Results**

*Steps 13 - ?*

---

The final step for our group is to submit the test data to the model generated earlier and view the results. Once we have the predictions, we will compare our values with the official

In [ ]:
#Step 13: Generate the predicitions

# The model we created earlier in Step 10 has a model. predict(input) capability
# Our input, just like model.fit, isn't testData,  but testDataEncoded instead
# We want to create a list of predicitions since there are known to be multiple hypothesis / premise pairs
testPredicitions = [jnp.argmax(i) for i in model.predict(testDataEncoded)]

In [ ]:
#Step 14: Save those predicitions to a submission file
        # This will contain the ID of the hypothesis / premise pair AND the predicition (2 columns)

submissionFile = testData.id.copy().to_frame() #Taking the id values from testData and copying them over to Row 1 (to_frame())
submissionFile['predicition'] = testPredicitions #Row 2 will be the testPredicitions we just found

In [ ]:
#Step 14.5: View the submissionFile and see what values are stored

submissionFile.head(len(trainData["id"].unique()))

,id,predicition
0,c6d58c3f69,2
1,cefcc82292,1
2,e98005252c,0
3,58518c10ba,2
4,c32b0d16df,1
...,...,...
5190,5f90dd59b0,1
5191,f357a04e86,1
5192,1f0ea92118,1
5193,0407b48afb,0


In [ ]:
#Extra 1

# Just to showcase what the official kaggle sample submission file looks like
# Note that the sample submission is filled with 1's, but contains the same format and the same number of items
exampleSubmission = pd.read_csv("CS301_Project_AI/sample_submission.csv")
exampleSubmission.head(len(exampleSubmission["id"].unique()))

,id,prediction
0,c6d58c3f69,1
1,cefcc82292,1
2,e98005252c,1
3,58518c10ba,1
4,c32b0d16df,1
...,...,...
5190,5f90dd59b0,1
5191,f357a04e86,1
5192,1f0ea92118,1
5193,0407b48afb,1
